In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster
@pyimport sklearn.metrics as metrics

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("/home/chuyu/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

data_info_all = []
for key in keys(data_info_dict)
    append!(data_info_all, data_info_dict[key])
end

In [ ]:
data_info_dict

In [ ]:
analyzer = "chuyu"

In [ ]:
function neighbor_percent(corr_map_pairwise_all_1, corr_map_pairwise_all_2, neighbor_percent_list)
    
    percent_stay_all_n = fill(NaN32, length(neighbor_percent_list), size(corr_map_pairwise_all_1, 1))
    for which_neuron = 1:size(corr_map_pairwise_all_1, 1)
        neighbor_rank_1 = reverse(sortperm(corr_map_pairwise_all_1[which_neuron,:]))
        neighbor_rank_2 = reverse(sortperm(corr_map_pairwise_all_2[which_neuron,:]))
        
        valid_neighbors_1 = findall(corr_map_pairwise_all_1[which_neuron,:].>=-1)
        valid_neighbors_2 = findall(corr_map_pairwise_all_2[which_neuron,:].>=-1)
        
        n_neighbors_list = floor.(Int32, neighbor_percent_list.*length(valid_neighbors_1))
        
        for (i_neighbor, n_neighbors) in enumerate(n_neighbors_list)
            top_neighbors_1 = intersect(neighbor_rank_1[1:n_neighbors], valid_neighbors_1)
            top_neighbors_2 = intersect(neighbor_rank_2[1:n_neighbors], valid_neighbors_2)
            

            percent_stay = sum(whether_in(top_neighbors_2,  top_neighbors_1))/length(top_neighbors_1)
            percent_stay_all_n[i_neighbor,which_neuron] = percent_stay
        end
    end 
    return percent_stay_all_n
end

# Process all data

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure10"

In [ ]:
key = "corner_cue_circle"

data_info_all = data_info_dict[key]
for which_data = 1:length(data_info_all)
data_info = data_info_all[which_data]

experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experimenter = data_info[5]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

    
file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
valid_neuron_1 = HDF5.readmmap(file["valid_neurons"])
close(file)


file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
all_files = readdir(file_folder_2)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_2, save_file_name)
file = h5open(info_filename, "r")
place_map_all_2 = HDF5.readmmap(file["place_map_all"])
valid_neuron_2 = HDF5.readmmap(file["valid_neurons"])
close(file)



save_file_name = "compare_map_results_original.h5"
compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(compare_filename, "r")
tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
close(file)


correct_index = whether_in(tel_place_cell_index, intersect(valid_neuron_1, valid_neuron_2))

# orientation-corrected background image and chamber roi image
cluster_file = h5open(joinpath(data_path(ds_save_cy_1), "cluster_analysis.h5"))
corr_map_pairwise_all_1_early = read(cluster_file,"corr_map_pairwise_all_1_early")[correct_index, correct_index]
corr_map_pairwise_all_1_late = read(cluster_file,"corr_map_pairwise_all_1_late")[correct_index, correct_index]
corr_map_pairwise_all_1 = read(cluster_file,"corr_map_pairwise_all_1")[correct_index, correct_index]
corr_map_pairwise_all_2 = read(cluster_file,"corr_map_pairwise_all_2")[correct_index, correct_index]
anatomy_pairwise_distance = read(cluster_file, "anatomy_pairwise_distance")[correct_index, correct_index]
close(cluster_file)
    


corr_map_pairwise_all_1[anatomy_pairwise_distance.<=20] .= -2
corr_map_pairwise_all_2[anatomy_pairwise_distance.<=20] .= -2


corr_map_pairwise_all_1_early[anatomy_pairwise_distance.<=20] .= -2
corr_map_pairwise_all_1_late[anatomy_pairwise_distance.<=20] .= -2;

neighbor_percent_list = numpy.linspace(0.02, 1, 50) 
percent_stay_all_n = neighbor_percent(corr_map_pairwise_all_1, corr_map_pairwise_all_2, neighbor_percent_list)
percent_stay_all_n_mean = numpy.mean(percent_stay_all_n, axis = 1);


percent_stay_all_n_earlylate = neighbor_percent(corr_map_pairwise_all_1_early, corr_map_pairwise_all_1_late, neighbor_percent_list)
percent_stay_all_n_mean_earlylate = numpy.mean(percent_stay_all_n_earlylate, axis = 1);





figure()
plot(neighbor_percent_list, percent_stay_all_n_mean, label= "s1 vs s2")
plot(neighbor_percent_list, percent_stay_all_n_mean_earlylate, label= "early vs late")
legend()

xlabel("k")
ylabel("Staying neighbors %")
    
title(key)
    
    
h5open(joinpath(data_path(ds_save_analyzer_1), "cluster_analysis_neighbors.h5"), "w") do file
    file["neighbor_percent_list"] = neighbor_percent_list
    file["percent_stay_all_n_mean_earlylate_percent"] = percent_stay_all_n_mean_earlylate
    file["percent_stay_all_n_mean_percent"] = percent_stay_all_n_mean;  
end;
        
        
end

In [ ]:
edge_neuron_nr = []
center_neuron_nr = []

key = "corner_cue_circle"

data_info_all = data_info_dict[key]
for which_data = 1:length(data_info_all)
data_info = data_info_all[which_data]

experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experimenter = data_info[5]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


save_file_name = "compare_map_results.h5"
compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(compare_filename, "r")
tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
peak_loc_map_1 = HDF5.readmmap(file["peak_all_1"])
peak_loc_map_2 = HDF5.readmmap(file["peak_all_2"])
peak_distance_edge_1_mm = HDF5.readmmap(file["peak_distance_edge_1_mm"])
close(file)

println(extrema(peak_distance_edge_1_mm))

median_distance_edge = 3
edge_neuron = confined_place_cell_index_1[peak_distance_edge_1_mm.<median_distance_edge]
center_neuron = confined_place_cell_index_1[peak_distance_edge_1_mm.>median_distance_edge]

edge_neuron_index = whether_in(tel_place_cell_index, edge_neuron)
center_neuron_index = whether_in(tel_place_cell_index, center_neuron)

    
println(length(edge_neuron))
  println(length(center_neuron))  
    
    
append!(edge_neuron_nr, length(edge_neuron))
append!(center_neuron_nr, length(center_neuron))
end

In [ ]:
mean(edge_neuron_nr)

In [ ]:
std(edge_neuron_nr)

In [ ]:
mean(center_neuron_nr)

In [ ]:
std(center_neuron_nr)

In [ ]:
key = "corner_cue_circle"

data_info_all = data_info_dict[key]
for which_data = 1:length(data_info_all)
data_info = data_info_all[which_data]

experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experimenter = data_info[5]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
valid_neuron_1 = HDF5.readmmap(file["valid_neurons"])
close(file)


file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
all_files = readdir(file_folder_2)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_2, save_file_name)
file = h5open(info_filename, "r")
place_map_all_2 = HDF5.readmmap(file["place_map_all"])
valid_neuron_2 = HDF5.readmmap(file["valid_neurons"])
close(file)



save_file_name = "compare_map_results_original.h5"
compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(compare_filename, "r")
tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
close(file)


correct_index = whether_in(tel_place_cell_index, intersect(valid_neuron_1, valid_neuron_2))
    
tel_place_cell_index = tel_place_cell_index[correct_index]

# orientation-corrected background image and chamber roi image
cluster_file = h5open(joinpath(data_path(ds_save_cy_1), "cluster_analysis.h5"))
corr_map_pairwise_all_1_early = read(cluster_file,"corr_map_pairwise_all_1_early")[correct_index, correct_index]
corr_map_pairwise_all_1_late = read(cluster_file,"corr_map_pairwise_all_1_late")[correct_index, correct_index]
corr_map_pairwise_all_1 = read(cluster_file,"corr_map_pairwise_all_1")[correct_index, correct_index]
corr_map_pairwise_all_2 = read(cluster_file,"corr_map_pairwise_all_2")[correct_index, correct_index]
anatomy_pairwise_distance = read(cluster_file, "anatomy_pairwise_distance")[correct_index, correct_index]
close(cluster_file)


save_file_name = "compare_map_results_original.h5"
compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(compare_filename, "r")
peak_distance_edge_1_mm = HDF5.readmmap(file["peak_distance_edge_1_mm"])
confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
close(file)


median_distance_edge = median(peak_distance_edge_1_mm)
edge_neuron = confined_place_cell_index_1[peak_distance_edge_1_mm.<median_distance_edge]
center_neuron = confined_place_cell_index_1[peak_distance_edge_1_mm.>median_distance_edge]

edge_neuron_index = whether_in(tel_place_cell_index, edge_neuron)
center_neuron_index = whether_in(tel_place_cell_index, center_neuron)


corr_map_pairwise_all_1[anatomy_pairwise_distance.<=20] .= -2
corr_map_pairwise_all_2[anatomy_pairwise_distance.<=20] .= -2

corr_map_pairwise_all_1_early[anatomy_pairwise_distance.<=20] .= -2
corr_map_pairwise_all_1_late[anatomy_pairwise_distance.<=20] .= -2;


corr_map_pairwise_all_1_chosen = corr_map_pairwise_all_1[edge_neuron_index, edge_neuron_index]
corr_map_pairwise_all_2_chosen = corr_map_pairwise_all_2[edge_neuron_index, edge_neuron_index]
corr_map_pairwise_all_1_early_chosen = corr_map_pairwise_all_1_early[edge_neuron_index, edge_neuron_index]
corr_map_pairwise_all_1_late_chosen = corr_map_pairwise_all_1_late[edge_neuron_index, edge_neuron_index]

neighbor_percent_list = numpy.linspace(0.02, 1, 50) 

    
percent_stay_all_n = neighbor_percent(corr_map_pairwise_all_1_chosen, corr_map_pairwise_all_2_chosen, neighbor_percent_list)
percent_stay_all_n_mean_edge = numpy.mean(percent_stay_all_n, axis = 1);


percent_stay_all_n_earlylate = neighbor_percent(corr_map_pairwise_all_1_early_chosen, corr_map_pairwise_all_1_late_chosen, neighbor_percent_list)
percent_stay_all_n_mean_earlylate_edge = numpy.mean(percent_stay_all_n_earlylate, axis = 1);






corr_map_pairwise_all_1_chosen = corr_map_pairwise_all_1[center_neuron_index, center_neuron_index]
corr_map_pairwise_all_2_chosen = corr_map_pairwise_all_2[center_neuron_index, center_neuron_index]
corr_map_pairwise_all_1_early_chosen = corr_map_pairwise_all_1_early[center_neuron_index, center_neuron_index]
corr_map_pairwise_all_1_late_chosen = corr_map_pairwise_all_1_late[center_neuron_index, center_neuron_index]


percent_stay_all_n = neighbor_percent(corr_map_pairwise_all_1_chosen, corr_map_pairwise_all_2_chosen, neighbor_percent_list)
percent_stay_all_n_mean_center = numpy.mean(percent_stay_all_n, axis = 1);


percent_stay_all_n_earlylate = neighbor_percent(corr_map_pairwise_all_1_early_chosen, corr_map_pairwise_all_1_late_chosen, neighbor_percent_list)
percent_stay_all_n_mean_earlylate_center = numpy.mean(percent_stay_all_n_earlylate, axis = 1);





figure()
plot(100 .*numpy.linspace(0.02, 1, 50), percent_stay_all_n_mean_edge, label= "s1 vs s2 edge", color = "r", linestyle="dashed")
plot(100 .*numpy.linspace(0.02, 1, 50), percent_stay_all_n_mean_earlylate_edge, label= "early vs late edge", color = "gray", linestyle="dashed")



plot(100 .*numpy.linspace(0.02, 1, 50), percent_stay_all_n_mean_center, label= "s1 vs s2 center", color = "r")
plot(100 .*numpy.linspace(0.02, 1, 50), percent_stay_all_n_mean_earlylate_center, label= "early vs late center", color = "gray")


legend()

xlabel("Neighbors %")
ylabel("Staying neighbors %")
    
title(key)
    
    
h5open(joinpath(data_path(ds_save_analyzer_1), "cluster_analysis_neighbors.h5"), "r+") do file
    file["edge_neuron_index"] = collect(edge_neuron_index)
    file["center_neuron_index"] = collect(center_neuron_index)
    file["percent_stay_all_n_mean_edge"] = percent_stay_all_n_mean_edge;
    file["percent_stay_all_n_mean_earlylate_edge"] = percent_stay_all_n_mean_earlylate_edge
        
    file["percent_stay_all_n_mean_center"] = percent_stay_all_n_mean_center;
    file["percent_stay_all_n_mean_earlylate_center"] = percent_stay_all_n_mean_earlylate_center
        
        
end;
    
end

In [ ]:
data_info_all = data_info_dict[key]
for which_data = 1:length(data_info_all)
data_info = data_info_all[which_data]

experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experimenter = data_info[5]
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
n_bins = save_file_name[end-4:end-3]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
valid_neuron_1 = HDF5.readmmap(file["valid_neurons"])
close(file)


file_folder_2 = joinpath(data_path(ds_save_cy_2), "")
all_files = readdir(file_folder_2)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_2, save_file_name)
file = h5open(info_filename, "r")
place_map_all_2 = HDF5.readmmap(file["place_map_all"])
valid_neuron_2 = HDF5.readmmap(file["valid_neurons"])
close(file)



save_file_name = "compare_map_results_original.h5"
compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(compare_filename, "r")
tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
close(file)


correct_index = whether_in(tel_place_cell_index, intersect(valid_neuron_1, valid_neuron_2))
    
tel_place_cell_index = tel_place_cell_index[correct_index]


# orientation-corrected background image and chamber roi image
cluster_file = h5open(joinpath(data_path(ds_save_cy_1), "cluster_analysis.h5"))
corr_map_pairwise_all_1_early = read(cluster_file,"corr_map_pairwise_all_1_early")[correct_index, correct_index]
corr_map_pairwise_all_1_late = read(cluster_file,"corr_map_pairwise_all_1_late")[correct_index, correct_index]
corr_map_pairwise_all_1 = read(cluster_file,"corr_map_pairwise_all_1")[correct_index, correct_index]
corr_map_pairwise_all_2 = read(cluster_file,"corr_map_pairwise_all_2")[correct_index, correct_index]
anatomy_pairwise_distance = read(cluster_file, "anatomy_pairwise_distance")[correct_index, correct_index]
close(cluster_file)


corr_map_pairwise_all_1[anatomy_pairwise_distance.<=20] .= -2
corr_map_pairwise_all_2[anatomy_pairwise_distance.<=20] .= -2


corr_map_pairwise_all_1_early[anatomy_pairwise_distance.<=20] .= -2
corr_map_pairwise_all_1_late[anatomy_pairwise_distance.<=20] .= -2;

neighbor_percent_list = numpy.linspace(0.02, 1, 50) 
percent_stay_all_n = neighbor_percent(corr_map_pairwise_all_1, corr_map_pairwise_all_2, neighbor_percent_list)
percent_stay_all_n_mean = numpy.mean(percent_stay_all_n, axis = 1);


percent_stay_all_n_earlylate = neighbor_percent(corr_map_pairwise_all_1_early, corr_map_pairwise_all_1_late, neighbor_percent_list)
percent_stay_all_n_mean_earlylate = numpy.mean(percent_stay_all_n_earlylate, axis = 1);



println(stats.wilcoxon(percent_stay_all_n[5,:], percent_stay_all_n_earlylate[5,:],alternative="less"))

percent_stay_all_n_mean_random_all = []
@showprogress for i_random in 1:1000
    random_idx = collect(1:size(corr_map_pairwise_all_1, 1))
    random_idx = numpy.random.permutation(random_idx)
    percent_stay_all_n_random = neighbor_percent(corr_map_pairwise_all_1, corr_map_pairwise_all_2[random_idx, random_idx], [neighbor_percent_list[5]])
    percent_stay_all_n_mean_random = numpy.mean(percent_stay_all_n_random, axis = 1);
    append!(percent_stay_all_n_mean_random_all, [percent_stay_all_n_mean_random])
end
println(neighbor_percent_list[5])
println(sum(mean(percent_stay_all_n[5,:]) .< [percent_stay_all_n_mean_random_all[i][1] for i in 1:length(percent_stay_all_n_mean_random_all)])/1000)
println(mean(percent_stay_all_n_mean_random_all)) 
    println(std(percent_stay_all_n_mean_random_all)) 

end